In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

num_attendees = 24
num_conflicts = 40

conflict_matrix = np.zeros((num_attendees, num_attendees), dtype=int)
conflicts = set()

while len(conflicts) < num_conflicts:
    i, j = np.random.randint(0, num_attendees, size=2)
    if i != j and (i, j) not in conflicts:
        conflicts.add((i, j))
        conflict_matrix[i, j] = 1
        if np.random.random() < 0.5:  # 50% chance of two-directional conflict
            conflict_matrix[j, i] = 1

conflict_tensor = torch.tensor(conflict_matrix, dtype=torch.float32)

In [ ]:
class SeatingNet(nn.Module):
    def __init__(self, num_people):
        super(SeatingNet, self).__init__()
        self.fc1 = nn.Linear(num_people * num_people, 256)
        # self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        # self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, num_people * num_people)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(1, -1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = x.view(num_attendees, num_attendees)
        x = self.softmax(x)
        return x

model = SeatingNet(num_attendees)

In [ ]:
def seating_loss(seating_probs, conflict_matrix):
    # Create adjacency matrix for the 6x4 seating arrangement
    adj_matrix = torch.zeros(24, 24)
    for i in range(24):
        if i % 4 != 3:  # Right neighbor
            adj_matrix[i, i+1] = 1
        if i % 4 != 0:  # Left neighbor
            adj_matrix[i, i-1] = 1
        if i < 20:  # Bottom neighbor
            adj_matrix[i, i+4] = 1
        if i >= 4:  # Top neighbor
            adj_matrix[i, i-4] = 1

    # Calculate probability of conflicts
    conflict_prob = torch.matmul(torch.matmul(seating_probs.T, adj_matrix), seating_probs)

    # Simplified loss: sum of conflict probabilities where there are actual conflicts
    loss = torch.sum(conflict_prob * conflict_matrix)

    # Add L2 regularization to encourage diverse seating
    l2_loss = torch.sum(seating_probs ** 2)

    return loss + 0.01 * l2_loss

optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    seating_probs = model(conflict_tensor)
    loss = seating_loss(seating_probs, conflict_tensor)
    loss.backward()

    # Gradient clipping
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Generate final seating arrangement
with torch.no_grad():
    seating_probs = model(conflict_tensor)
    seating = torch.argmax(seating_probs, dim=1)
    print("Final seating arrangement:", seating.numpy())

# Evaluate the solution
def evaluate_seating(seating, conflict_matrix):
    conflicts = 0
    for i in range(num_attendees):
        for j in range(num_attendees):
            if conflict_matrix[i, j] == 1:
                if abs(seating[i] - seating[j]) == 1 or abs(seating[i] - seating[j]) == 4:
                    conflicts += 1
    return conflicts

final_conflicts = evaluate_seating(seating, conflict_tensor)
print(f"Number of conflicts in final arrangement: {final_conflicts}")

Epoch [100/1000], Loss: 0.0320
Epoch [200/1000], Loss: 0.0289
Epoch [300/1000], Loss: 0.0276
Epoch [400/1000], Loss: 0.0263
Epoch [500/1000], Loss: 0.0257
Epoch [600/1000], Loss: 0.0253
Epoch [700/1000], Loss: 0.0248
Epoch [800/1000], Loss: 0.0246
Epoch [900/1000], Loss: 0.0243
Epoch [1000/1000], Loss: 0.0242
Final seating arrangement: [15  7 15 19 15 22  7 22 19 20 22 22 15  5 15 19 18 15 23 18 15  7 23  3]
Number of conflicts in final arrangement: 11
